In [ ]:
import sys
import cPickle as pickle
import numpy as np
import time

data_path = 'data/linear_svm.pkl'
with open(data_path, 'r') as f:
    datadict = pickle.load(f)
    X = datadict['X']
    y = datadict['y']

W = np.random.randn(10, 3073) * 0.0001 

%load_ext autoreload
%autoreload 2

In [ ]:
from files.linear_svm import svm_loss_naive
from files.gradient_check import grad_check

loss, grad = svm_loss_naive(W, X, y, 0)
f = lambda w: svm_loss_naive(w, X, y, 0.0)[0]

grad_numerical = grad_check(f, W, grad, 10)

In [ ]:
start = time.time()
loss_naive, grad_naive = svm_loss_naive(W, X, y, 0.00001)
stop = time.time()
naive_time = stop - start

print 'Naive loss: %e computed in %fs' % (loss_naive, stop - start)

start = time.time()
loss_vectorized, _ = svm_loss_vectorized(W, X, y, 0.00001)
stop = time.time()
vectorized_time = stop - start

print 'Vectorized loss: %e computed in %fs' % (loss_vectorized, stop - start)

# The loss difference should be 0, but your vectorized implementation should be much faster.
print 'difference: %f. Vectorized version is %fx faster' % (loss_naive - loss_vectorized, naive_time/vectorized_time)